<a href="https://colab.research.google.com/github/luanvu2307/AI-Hackathon-2020/blob/main/AI4VN_hackathon_MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Tổng quan

Hướng dẫn này dựa trên hướng dẫn [Transfer learning and fine-tuning](https://www.tensorflow.org/tutorials/images/transfer_learning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! git clone https://github.com/hcmcaic/ai4vn-hackathon-2020.git ai4vn

Cloning into 'ai4vn'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 20 (delta 2), reused 6 (delta 2), pack-reused 14
Unpacking objects: 100% (20/20), done.


In [ ]:
% cd ai4vn

/content/ai4vn


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from dataloader import image_dataset_from_directory

In [ ]:
!unzip ai4vn_2020.zip

Archive:  ai4vn_2020.zip
   creating: ai4vn_2020/
   creating: ai4vn_2020/sample_data/
   creating: ai4vn_2020/sample_data/1/
  inflating: ai4vn_2020/sample_data/1/001500.jpg  
 extracting: ai4vn_2020/sample_data/1/001501.jpg  
  inflating: ai4vn_2020/sample_data/1/001502.png  
  inflating: ai4vn_2020/sample_data/1/001503.jpeg  
  inflating: ai4vn_2020/sample_data/1/001504.jpg  
  inflating: ai4vn_2020/sample_data/1/001505.jpg  
  inflating: ai4vn_2020/sample_data/1/001506.jpg  
  inflating: ai4vn_2020/sample_data/1/001507.jpg  
  inflating: ai4vn_2020/sample_data/1/001508.jpg  
  inflating: ai4vn_2020/sample_data/1/001509.jpg  
 extracting: ai4vn_2020/sample_data/1/001510.jpg  
 extracting: ai4vn_2020/sample_data/1/001511.png  
  inflating: ai4vn_2020/sample_data/1/001512.jpg  
  inflating: ai4vn_2020/sample_data/1/001513.png  
 extracting: ai4vn_2020/sample_data/1/001514.jpg  
  inflating: ai4vn_2020/sample_data/1/001515.jpg  
  inflating: ai4vn_2020/sample_data/1/001516.jpg  
 extra

## Tiền xử lý dữ liệu

### Tải dữ liệu

In [ ]:
# move test_data to test folder to use image_dataset_from_directory function
! mkdir ai4vn_2020/test && mv ai4vn_2020/test_data ai4vn_2020/test

In [ ]:
# ! unzip sample_data.zip

path_to_data = 'ai4vn_2020/sample_data'


BATCH_SIZE = 10
IMG_SIZE = (160, 160)

NUM_CLASS = 7

train_dataset, train_dataset_filenames = image_dataset_from_directory(path_to_data,
                                             validation_split=0.2,
                                             subset="training",
                                             shuffle=True,
                                             seed=505,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

Found 210 files belonging to 7 classes.
Using 168 files for training.


In [ ]:
validation_dataset, validation_dataset_filenames = image_dataset_from_directory(path_to_data,
                                                  validation_split=0.2,
                                                  subset="training",
                                                  shuffle=True,
                                                  seed=505,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Found 210 files belonging to 7 classes.
Using 168 files for training.


In [ ]:
test_dataset, test_dataset_filenames = image_dataset_from_directory('ai4vn_2020/test',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

Found 20 files belonging to 1 classes.


In [ ]:
type(test_dataset_filenames)

list

In [ ]:
validation_dataset.class_names

Hiển thị 9 hình ảnh và nhãn đầu tiên từ bộ huấn luyện:

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

### Định cấu hình cho tập dữ liệu

Use buffered prefetching to load images from disk without having I/O become blocking. To learn more about this method see the [data performance](https://www.tensorflow.org/guide/data_performance) guide.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)



### Sử dụng tăng cường dữ liệu

Khi dữ liệu ảnh không đủ lớn, bạn có thể tăng tính đa dạng của các mẫu dữ liệu bằng cách áp các phép biến đổi ngẫu nhiên. Điều này giúp giảm [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit). Bạn có thể tìm hiểu thêm về tăng dữ liệu trong [hướng dẫn](https://www.tensorflow.org/tutorials/images/data_augmentation) này.

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

Dưới đây là kết quả của việc tăng cường dữ liệu trên một tấm ảnh.

In [ ]:
for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Rescale pixel values
Trong ví dụ này, ta sử dụng mô hình cơ sở tf.keras.applications.MobileNetV2. Mô hình này sử dụng giá trị điểm ảnh trong đoạn `[-1,1]` nhưng giá trị ảnh hiện tại đang trong đoạn `[0-255]`. Vì thế ta cần chuẩn hóa lại miền giá trị điểm ảnh.

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)


## Create the base model from the pre-trained convnets


In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


Mô hình trích xuất đặc trưng này nhận tấm ảnh với kích thước `160x160x3` chuyển thành đặc trưng `5x5x1280`

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

## Feature extraction

### Freeze the convolutional base

In [ ]:
base_model.trainable = False

In [ ]:
# Let's take a look at the base model architecture
base_model.summary()

### Add a classification head

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = tf.keras.layers.Dense(NUM_CLASS)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

### Compile the model


In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Train the model


In [ ]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

### Learning curves


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## Fine tuning


### Un-freeze the top layers of the model


In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

### Compile the model



In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

### Continue training the model

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Evaluation and prediction

In [ ]:
loss, accuracy = model.evaluate(validation_dataset)
print('Validation accuracy :', accuracy)

In [ ]:
#Retrieve a batch of images from the test set
image_batch, label_batch = validation_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)

predicted_class = np.argmax(predictions, axis=-1)


print('Predictions:\n', predicted_class)
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predicted_class[i]])
  plt.axis("off")

In [ ]:
predictions = model.predict(test_dataset)

predicted_class = np.argmax(predictions, axis=-1)


print('Predictions:\n', predicted_class)

In [ ]:
with open('submission.txt', 'w') as submission_file:
  for filename, predicted in zip(test_dataset_filenames, predicted_class):
    submission_file.write('{}\t{}\n'.format(filename.split('/')[-1], class_names[predicted]))